In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import folium

In [2]:
from google.colab import drive

drive._mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
df=pd.read_csv('/content/drive/MyDrive/WeatherEvents_Jan2016-Dec2022 new.csv')

In [4]:
df = df.dropna()

In [5]:
datetimeFormat = '%Y-%m-%d %H:%M:%S'
df['End']=pd.to_datetime(df['EndTime(UTC)'], format=datetimeFormat)
df['Start']=pd.to_datetime(df['StartTime(UTC)'], format=datetimeFormat)
df['Duration']=df['End']-df['Start']
df['Duration'] = df['Duration'].dt.total_seconds()
df['Duration'] = df['Duration'] / (60*60) # Converting to hours
df = df[(df['Duration']< 30*24) & (df['Duration'] > 0)] # More than 30 days or less than 0 hours discard.
df.tail(3)

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,End,Start,Duration
8627178,W-9091892,Cold,Severe,2022-12-21 23:53:00,2022-12-24 02:53:00,0.0,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0,2022-12-24 02:53:00,2022-12-21 23:53:00,51.0
8627179,W-9091893,Cold,Severe,2022-12-24 03:53:00,2022-12-24 07:53:00,0.0,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0,2022-12-24 07:53:00,2022-12-24 03:53:00,4.0
8627180,W-9091894,Cold,Severe,2022-12-24 09:53:00,2022-12-24 11:53:00,0.0,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0,2022-12-24 11:53:00,2022-12-24 09:53:00,2.0


In [6]:
df = df[df.Duration < 20]

In [7]:
df.shape

(8541296, 17)

In [8]:
df4=df[df['Severity']=='Severe']

In [9]:
df4.shape

(1698812, 17)

In [10]:
df4.head(3)

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,End,Start,Duration
6,W-7,Fog,Severe,2016-01-09 12:54:00,2016-01-09 15:34:00,0.0,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-09 15:34:00,2016-01-09 12:54:00,2.666667
8,W-9,Fog,Severe,2016-01-09 16:14:00,2016-01-09 16:34:00,0.0,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-09 16:34:00,2016-01-09 16:14:00,0.333333
10,W-11,Cold,Severe,2016-01-09 16:54:00,2016-01-09 20:34:00,0.0,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-09 20:34:00,2016-01-09 16:54:00,3.666667


In [11]:
df2 = df.groupby(['AirportCode','City','State',
                  'LocationLat', 'LocationLng','Type', 'Severity']).agg({'Duration':['sum']}).reset_index()
df2.columns = pd.MultiIndex.from_tuples((("AirportCode", " "),("City", " "),
                                         ("State", " "), ("LocationLat", " "),
                                         ("LocationLng", " "), ("Type", " "), ("Severity", " "),
                                         ("Duration", " ")))
df2.columns = df2.columns.get_level_values(0)
df2['Duration'] = df2['Duration'] / (24*6*3.65) # Yearly Percentage
df2 = df2.sort_values(by='Duration')
df2.tail(3)

,AirportCode,City,State,LocationLat,LocationLng,Type,Severity,Duration
18629,KSXT,Wolf Creek,OR,42.6000,-123.3656,Fog,Severe,16.915874
19597,KUIL,Forks,WA,47.9375,-124.5550,Rain,Light,17.405758
17443,KSBD,San Bernardino,CA,34.0903,-117.2471,Fog,Moderate,17.894597


In [12]:
df2.shape

(20565, 8)

In [13]:
df2.shape

(20565, 8)

In [14]:
south = ['AL', 'GA', 'FL', 'SC', 'NC', 'TN', 'MS', 'LA', 'AR', 'OK', 'TX']
atlantic = ['KY', 'VA', 'WV', 'MD', 'DE', 'PA', 'NJ']
new_england = ['NY', 'CT', 'RI', 'MA', 'VT', 'NH', 'ME']
midwest = ['OH', 'IN', 'MI', 'IL', 'WI', 'IA', 'MO', 'MN', 'NE', 'KS', 'SD', 'ND']
west = ['MT', 'WY', 'ID', 'NV', 'UT', 'CO', 'AZ', 'NM']
pacific = ['AK', 'WA', 'OR', 'CA', 'HI']
df2.loc[df2['State'].isin(south), 'Region'] = 'South'
df2.loc[df2['State'].isin(atlantic), 'Region'] = 'Atlantic'
df2.loc[df2['State'].isin(new_england), 'Region'] = 'New England'
df2.loc[df2['State'].isin(midwest), 'Region'] = 'Midwest'
df2.loc[df2['State'].isin(west), 'Region'] = 'West'
df2.loc[df2['State'].isin(pacific), 'Region'] = 'Pacific'
df2 = df2[['AirportCode', 'City', 'State', 'Region', 'LocationLat', 'LocationLng', 'Type', 'Severity', 'Duration']]
df2.tail(3)

,AirportCode,City,State,Region,LocationLat,LocationLng,Type,Severity,Duration
18629,KSXT,Wolf Creek,OR,Pacific,42.6000,-123.3656,Fog,Severe,16.915874
19597,KUIL,Forks,WA,Pacific,47.9375,-124.5550,Rain,Light,17.405758
17443,KSBD,San Bernardino,CA,Pacific,34.0903,-117.2471,Fog,Moderate,17.894597


In [15]:
pre_flat = df.groupby(['AirportCode','City','State',
                  'LocationLat', 'LocationLng','Type']).agg({'Duration':['sum']}).reset_index()
pre_flat.columns = pd.MultiIndex.from_tuples((("AirportCode", " "),("City", " "),
                                              ("State", " "), ("LocationLat", " "),
                                              ("LocationLng", " "), ("Type", " "),
                                              ("Duration", " ")))
pre_flat.columns = pre_flat.columns.get_level_values(0)
pre_flat['Duration'] = pre_flat['Duration'] / (24*6*3.65) # Yearly Percentage
pre_flat = pre_flat.sort_values(by='Duration')
pre_flat.tail(3)

,AirportCode,City,State,LocationLat,LocationLng,Type,Duration
10684,KSXT,Wolf Creek,OR,42.6000,-123.3656,Fog,18.629693
280,K3TH,Thompson Falls-West End,MT,47.6000,-115.3667,Snow,19.111396
11253,KUIL,Forks,WA,47.9375,-124.5550,Rain,20.296423


In [16]:
df_flat = pre_flat.pivot_table(index='AirportCode', columns='Type', values=['Duration']).reset_index().fillna(0)
df_flat.columns = pd.MultiIndex.from_tuples(((' ', 'AirportCode'),(' ', 'Cold'),(' ', 'Fog'),
            (' ',  'Hail'),(' ', 'Precipitation'),(' ', 'Rain'),(' ', 'Snow'),(' ', 'Storm')))
df_flat.columns = df_flat.columns.get_level_values(1)
uniqueKey = df2[['AirportCode', 'City',
                 'State', 'Region', 'LocationLat', 'LocationLng']].sort_values(by='AirportCode').drop_duplicates()
weather = pd.merge(df_flat, uniqueKey, how='inner', on='AirportCode')
weather.tail(3)

,AirportCode,Cold,Fog,Hail,Precipitation,Rain,Snow,Storm,City,State,Region,LocationLat,LocationLng
2051,KYNG,0.013128,2.927987,0.011035,0.108733,11.310724,7.547438,0.003647,Vienna,OH,Midwest,41.2544,-80.6739
2052,KZPH,0.643709,2.014206,0.000000,0.058980,5.163623,0.000000,0.001903,Zephyrhills,FL,South,28.2281,-82.1559
2053,KZZV,0.126490,2.372780,0.000000,0.105308,7.234970,2.205226,0.000000,Zanesville,OH,Midwest,39.9444,-81.8921


In [17]:
df4_agg = (
    df4
    .groupby([
        'AirportCode','City','State','LocationLat','LocationLng','Type'
    ], as_index=False)['Duration']
    .sum()
)

# Convert to “yearly percentage” matching your prior pipeline:
df4_agg['Duration'] = df4_agg['Duration'] / (24 * 6 * 3.65)


# ── 2) Assign U.S. regions ──────────────────────────────────────────────────────

south      = ['AL','GA','FL','SC','NC','TN','MS','LA','AR','OK','TX']
atlantic   = ['KY','VA','WV','MD','DE','PA','NJ']
new_england= ['NY','CT','RI','MA','VT','NH','ME']
midwest    = ['OH','IN','MI','IL','WI','IA','MO','MN','NE','KS','SD','ND']
west       = ['MT','WY','ID','NV','UT','CO','AZ','NM']
pacific    = ['AK','WA','OR','CA','HI']

def assign_region(st):
    if st in south:       return 'South'
    if st in atlantic:    return 'Atlantic'
    if st in new_england: return 'New England'
    if st in midwest:     return 'Midwest'
    if st in west:        return 'West'
    if st in pacific:     return 'Pacific'
    return 'Other'

df4_agg['Region'] = df4_agg['State'].apply(assign_region)


In [18]:
fig_sev_hist = px.histogram(
    df4_agg,
    x='Type',
    y='Duration',
    histfunc='avg',
    color='Region',
    color_discrete_sequence=px.colors.qualitative.Safe,
    title='Avg Yearly Duration of Severe Weather Events by Type'
)
fig_sev_hist.update_xaxes(categoryorder='mean descending')
fig_sev_hist.update_yaxes(title_text='Mean Duration (% per Year)')
fig_sev_hist.update_layout(height=750, width=1000)

fig_sev_hist.show()
fig_sev_hist.write_html("severe_weather_duration.html")

In [19]:

df4_pivot = (
    df4_agg
    .pivot_table(
        index=['AirportCode','City','State','Region','LocationLat','LocationLng'],
        columns='Type',
        values='Duration',
        fill_value=0    # fill any existing cells with 0
    )
    .reset_index()
)

# Ensure all weather-types appear as columns, even if zero
all_types = ['Cold','Fog','Hail','Precipitation','Rain','Snow','Storm']
for t in all_types:
    if t not in df4_pivot.columns:
        df4_pivot[t] = 0.0

# (Optional) Reorder columns so that the Type columns come in a known order
df4_pivot = df4_pivot[
    ['AirportCode','City','State','Region','LocationLat','LocationLng'] + all_types
]

In [20]:
df_rain_high = df4_pivot[df4_pivot['Storm'] > 0.5]

# 2) Build the geo‐scatter on that subset
fig_sev_rain = px.scatter_geo(
    df_rain_high,
    lat='LocationLat',
    lon='LocationLng',
    color='Storm',
    color_continuous_scale=px.colors.sequential.Cividis,
    hover_name=df_rain_high['City'] + ', ' + df_rain_high['State'],
    scope='usa',
    title='Severe Events: City-Level Stormy Days % (> 0.5% Yearly)'
)
fig_sev_rain.update_layout(height=750, width=1000)

# 3) Show & save
fig_sev_rain.show()
fig_sev_rain.write_html("severe_city_storm_gt05.html")

In [21]:
cb_palette = px.colors.qualitative.Safe

# 2) Build the histogram with that palette
fig_sum = px.histogram(
    df2,
    x='Type',
    y='Duration',
    histfunc='avg',
    color='Severity',
    color_discrete_sequence=cb_palette,
    title='Nation Wide Weather Events Duration'
)

# 3) Tidy up axes and sizing
fig_sum.update_xaxes(categoryorder='mean descending')
fig_sum.update_yaxes(title_text='Mean of Duration % per Year')
fig_sum.update_layout(height=750, width=1000)

# 4a) Show interactively in your notebook or app
fig_sum.show()

# 4b) Save as a standalone HTML (retains full interactivity)
fig_sum.write_html("weather_duration.html")


In [22]:
fig_sum = px.histogram(df2, x='Type', y= 'Duration',  histfunc = 'avg', color='Severity',
                      title = 'Nation Wide Weather Events Duration')
fig_sum.update_xaxes(categoryorder='mean descending')
fig_sum.update_yaxes(title_text='Mean of Duration % per Year')
fig_sum.update_layout(height=750, width=1000)
fig_sum.show()

In [23]:
df.shape

(8541296, 17)

In [24]:
df2.shape

(20565, 9)

In [25]:
import numpy as np

# Create 4 bins: Low / Medium-Low / Medium-High / High
weather['RainBin'] = pd.qcut(weather['Rain'], 4, labels=[
    'Low', 'Med-Low', 'Med-High', 'High'
])

fig = px.scatter_geo(
    weather,
    lat='LocationLat',
    lon='LocationLng',
    color='RainBin',
    # use a discrete Safe palette
    color_discrete_sequence=px.colors.qualitative.Safe,
    hover_name=weather['City'] + ', ' + weather['State'],
    scope='usa',
    title='City-Wide Rain % (binned)'
)
fig.update_layout(height=750, width=1000)
fig.show()


In [26]:
cb_seq = px.colors.sequential.Viridis

# 2) Build the scatter_geo with that palette
fig_rain = px.scatter_geo(
    weather,
    lat='LocationLat',
    lon='LocationLng',
    color='Rain',
    color_continuous_scale=cb_seq,
    hover_name=weather['City'] + ', ' + weather['State'],
    scope='usa',
    title='City Wide Rainy Days Percentage Each Year from 2016 to 2021'
)

# 3) (Optional) Resize for consistency
fig_rain.update_layout(height=750, width=1000)

# 4a) Show interactively
fig_rain.show()

# 4b) Save as standalone HTML (retains pan/zoom/hover)
fig_rain.write_html("city_rain_percent.html")

In [27]:
fig_rain = px.scatter_geo(weather, lat='LocationLat', lon='LocationLng',
                      color="Rain",
                      hover_name=weather['City'] + ', ' + weather['State'],
                      color_continuous_scale='dense',
                      scope="usa",
                      title ='Fig 5. City Wide Rainy Days Percentage Each Year from 2016 to 2021')
fig_rain.show()

In [28]:
fig_fog = px.scatter_geo(
    weather,
    lat='LocationLat',
    lon='LocationLng',
    color='Fog',
    color_continuous_scale=cb_seq,
    hover_name=weather['City'] + ', ' + weather['State'],
    scope='usa',
    title='City Wide Foggy Days Percentage Each Year from 2016 to 2021'
)

# 3) Resize for consistency
fig_fog.update_layout(height=750, width=1000)

# 4a) Show interactively
fig_fog.show()

# 4b) Save as standalone HTML (retains full interactivity)
fig_fog.write_html("city_fog_percent.html")

In [29]:
fig_fog = px.scatter_geo(weather, lat='LocationLat', lon='LocationLng',
                      color="Fog",
                      hover_name=weather['City'] + ', ' + weather['State'],
                      color_continuous_scale='dense',
                      scope="usa",
                      title ='Fig 6. City Wide Foggy Days Percentage Each Year from 2016 to 2021')
fig_fog.show()

In [30]:
from sklearn.cluster import KMeans

X = df_flat.drop(['AirportCode','Cold', 'Hail'], axis=1)

In [31]:
df_flat.shape

(2054, 8)

In [32]:
from sklearn.cluster import KMeans
distortions = []

K = range(1,20)
for k in K:
    kmean = KMeans(n_clusters=k, random_state=0, n_init = 50, max_iter = 500)
    kmean.fit(X)
    distortions.append(kmean.inertia_)


In [33]:
kmeans = KMeans(n_clusters=4, random_state=0).fit(X)

df_flat['Cluster'] = (kmeans.labels_).astype(str)
df_cluster = pd.merge(df_flat[['AirportCode','Cluster']], weather.drop(['Cold','Hail'], axis=1),
                      how='inner', on='AirportCode')

In [34]:
fig_cluster = px.scatter_geo(
    df_cluster,
    lat='LocationLat',
    lon='LocationLng',
    color='Cluster',
    color_discrete_sequence=cb_palette,
    hover_name=weather['City'] + ', ' + weather['State'],
    scope='usa',
    title='Fig 11. City Wide Weather Cluster Distribution'
)

# 3) Resize for visual consistency
fig_cluster.update_layout(height=750, width=1000)

# 4a) Display interactively
fig_cluster.show()

# 4b) Save as standalone HTML (full interactivity)
fig_cluster.write_html("weather_cluster_distribution.html")

In [35]:
# Group by Cluster and get average weather values
cluster_weather_summary = df_cluster.groupby('Cluster').mean(numeric_only=True)

# Display the full summary
print(cluster_weather_summary)

              Fog  Precipitation      Rain      Snow     Storm  LocationLat  \
Cluster                                                                       
0        3.087023       0.072450  9.010153  1.516417  0.011760    40.215756   
1        1.575055       0.064847  2.021808  0.834553  0.094241    37.346052   
2        2.251712       0.047232  4.939154  5.146267  0.088158    44.173449   
3        2.407336       0.113619  5.680425  0.561099  0.013783    35.173068   

         LocationLng  
Cluster               
0         -86.298313  
1        -103.418972  
2         -94.104233  
3         -90.251804  


In [36]:
fig_cluster = px.scatter_geo(df_cluster, lat='LocationLat', lon='LocationLng',
                      hover_name=weather['City'] + ', ' + weather['State'],
                      scope="usa",
                      color_discrete_sequence =['#AB63FA', '#EF553B', '#00CC96','#636EFA'],
                      color = 'Cluster',
                      title ='Fig 11. City Wide Weather Cluster Distribution')
fig_cluster.show()